In [29]:
import ipyleaflet as ipyl
import ipywidgets as ipyw
import json
import pygeohash as gh
from ipyleaflet import (
    Map, TileLayer, SplitMapControl
)

In [131]:
map_l = ipyl.Map(center=[53.88, 27.45], zoom=0)
map_r = ipyl.Map(center=[53.88, 27.45], zoom=0)

# geojson layer with hover handler
with open('./europe_110.geo.json') as f:
    data = json.load(f)
for feature in data['features']:
    feature['properties']['style'] = {
        'color': 'grey',
        'weight': 1,
        'fillColor': 'grey',
        'fillOpacity': 0.2
    }
    feature['properties']['geometry'] = feature['geometry']
    
layer = ipyl.GeoJSON(data=data, hover_style={'fillColor': 'red'})

count = 0    

def click_handler(event=None, properties=None, id=None, **args):
    if properties is not None:
        geometry = properties['geometry']
        if geometry['type'] == 'Polygon':
            polygon = ipyl.Polygon(locations=[[p[1], p[0]] for p in geometry['coordinates'][0]], color='yellow', fillColor='yellow')
            map_l.add_layer(polygon)
    if len(args.keys()) > 0:
        lat = args['coordinates'][0]
        lon = args['coordinates'][1]
    
layer.on_click(click_handler)
map_l.add_layer(layer)
left = TileLayer(url='https://cartodb-basemaps-{s}.global.ssl.fastly.net/light_all/{z}/{x}/{y}.png')
right = TileLayer(url='https://cartodb-basemaps-{s}.global.ssl.fastly.net/dark_all/{z}/{x}/{y}.png')

# Create a callback for when the center of the map has changed
def map_l_on_center_change(change):
    new_left_center = change['new']
    map_r.center = new_left_center

    # Create a callback for when the center of the map has changed
def map_l_on_zoom_change(zoom):
    new_left_zoom = zoom['new']
    map_r.zoom = new_left_zoom
    
map_l.observe(map_l_on_center_change, names='center')
map_l.observe(map_l_on_zoom_change, names='zoom')

draw_control = ipyl.DrawControl()
map_r.add_control(draw_control)
map_r.add_control(ipyl.LayersControl())

In [132]:

ipyw.VBox([map_l, map_r])